# Loop 17 Strategic Analysis

## Current Status
- Best CV/LB: 70.630465 (from snapshot 21180223864)
- Target: 68.919154
- Gap: 1.711 (2.42%)

## Key Findings After 18 Experiments
1. ALL valid solutions converge to ~70.630
2. Tessellation approaches produce WORSE scores than baseline
3. SA optimizers (bbox3, sa_v1_parallel) produce overlapping trees
4. Tree removal gives only 0.00001 improvement
5. Basin hopping, GA, constraint programming all failed

In [ ]:
import pandas as pd
import numpy as np
import os

# Load current best
df = pd.read_csv('/home/code/exploration/datasets/ensemble_best.csv')
print(f"Current best rows: {len(df)}")

# Check score distribution by N
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon

getcontext().prec = 25
scale_factor = Decimal("1e15")

class ChristmasTree:
    def __init__(self, center_x="0", center_y="0", angle="0"):
        self.center_x = Decimal(center_x)
        self.center_y = Decimal(center_y)
        self.angle = Decimal(angle)

        trunk_w = Decimal("0.15")
        trunk_h = Decimal("0.2")
        base_w = Decimal("0.7")
        mid_w = Decimal("0.4")
        top_w = Decimal("0.25")
        tip_y = Decimal("0.8")
        tier_1_y = Decimal("0.5")
        tier_2_y = Decimal("0.25")
        base_y = Decimal("0.0")
        trunk_bottom_y = -trunk_h

        initial_polygon = Polygon([
            (Decimal("0.0") * scale_factor, tip_y * scale_factor),
            (top_w / Decimal("2") * scale_factor, tier_1_y * scale_factor),
            (top_w / Decimal("4") * scale_factor, tier_1_y * scale_factor),
            (mid_w / Decimal("2") * scale_factor, tier_2_y * scale_factor),
            (mid_w / Decimal("4") * scale_factor, tier_2_y * scale_factor),
            (base_w / Decimal("2") * scale_factor, base_y * scale_factor),
            (trunk_w / Decimal("2") * scale_factor, base_y * scale_factor),
            (trunk_w / Decimal("2") * scale_factor, trunk_bottom_y * scale_factor),
            (-(trunk_w / Decimal("2")) * scale_factor, trunk_bottom_y * scale_factor),
            (-(trunk_w / Decimal("2")) * scale_factor, base_y * scale_factor),
            (-(base_w / Decimal("2")) * scale_factor, base_y * scale_factor),
            (-(mid_w / Decimal("4")) * scale_factor, tier_2_y * scale_factor),
            (-(mid_w / Decimal("2")) * scale_factor, tier_2_y * scale_factor),
            (-(top_w / Decimal("4")) * scale_factor, tier_1_y * scale_factor),
            (-(top_w / Decimal("2")) * scale_factor, tier_1_y * scale_factor),
        ])
        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(
            rotated,
            xoff=float(self.center_x * scale_factor),
            yoff=float(self.center_y * scale_factor),
        )

def calculate_score(trees):
    xys = np.concatenate([np.asarray(t.polygon.exterior.xy).T / 1e15 for t in trees])
    min_x, min_y = xys.min(axis=0)
    max_x, max_y = xys.max(axis=0)
    score = max(max_x - min_x, max_y - min_y) ** 2 / len(trees)
    return score

def load_trees(n, df):
    group_data = df[df["id"].str.startswith(f"{n:03d}_")]
    trees = []
    for _, row in group_data.iterrows():
        x = str(row["x"]).lstrip('s')
        y = str(row["y"]).lstrip('s')
        deg = str(row["deg"]).lstrip('s')
        trees.append(ChristmasTree(x, y, deg))
    return trees

# Calculate scores by N
scores = {}
for n in range(1, 201):
    trees = load_trees(n, df)
    scores[n] = calculate_score(trees)

print(f"Total score: {sum(scores.values()):.6f}")
print(f"\nTop 10 worst N values (highest score):")
worst = sorted(scores.items(), key=lambda x: -x[1])[:10]
for n, s in worst:
    print(f"  N={n}: {s:.6f}")

In [ ]:
# Analyze packing efficiency
print("\nPacking Efficiency Analysis:")
print("="*60)

# Tree area (approximate)
tree_area = 0.7 * 0.5 + 0.4 * 0.25 + 0.25 * 0.3 + 0.15 * 0.2  # rough estimate
print(f"Approximate tree area: {tree_area:.4f}")

# For each N, calculate efficiency
efficiencies = {}
for n, score in scores.items():
    # score = side^2 / n, so side = sqrt(score * n)
    side = np.sqrt(score * n)
    box_area = side ** 2
    tree_total_area = n * tree_area
    efficiency = tree_total_area / box_area
    efficiencies[n] = efficiency

print(f"\nAverage efficiency: {np.mean(list(efficiencies.values())):.4f}")
print(f"Min efficiency: {min(efficiencies.values()):.4f} at N={min(efficiencies, key=efficiencies.get)}")
print(f"Max efficiency: {max(efficiencies.values()):.4f} at N={max(efficiencies, key=efficiencies.get)}")

# Find N values with low efficiency (potential for improvement)
print("\nN values with lowest efficiency (most room for improvement):")
low_eff = sorted(efficiencies.items(), key=lambda x: x[1])[:10]
for n, eff in low_eff:
    print(f"  N={n}: efficiency={eff:.4f}, score={scores[n]:.6f}")

In [ ]:
# Check what the theoretical minimum score would be
print("\nTheoretical Analysis:")
print("="*60)

# If we could pack trees perfectly (no wasted space)
# Total tree area = N * tree_area
# Minimum box side = sqrt(N * tree_area)
# Minimum score = (sqrt(N * tree_area))^2 / N = tree_area

print(f"Theoretical minimum score per N (if perfect packing): {tree_area:.6f}")
print(f"Theoretical minimum total score: {200 * tree_area:.6f}")

# But trees can't pack perfectly due to shape
# Let's estimate the actual minimum based on best N values
best_efficiency = max(efficiencies.values())
print(f"\nBest observed efficiency: {best_efficiency:.4f}")
print(f"Estimated achievable minimum score per N: {tree_area / best_efficiency:.6f}")
print(f"Estimated achievable total: {200 * tree_area / best_efficiency:.6f}")

# Compare to target
print(f"\nTarget score: 68.919154")
print(f"Current score: {sum(scores.values()):.6f}")
print(f"Gap: {sum(scores.values()) - 68.919154:.6f}")

In [ ]:
# Analyze the jiweiliu kernel approach
print("\nJiweiliu Kernel Analysis:")
print("="*60)
print("""
The jiweiliu kernel uses a 2-TREE SEED CONFIGURATION:
- Two trees at specific positions with specific angles
- These are then translated in a grid pattern
- SA optimizes the translation parameters (a, b) and seed positions

Initial seeds:
  Tree 1: (-4.19, -4.50, 74.54°)
  Tree 2: (-4.92, -4.73, 254.54°)

Initial translations:
  a = 0.874 (x direction)
  b = 0.750 (y direction)

This creates a FUNDAMENTALLY DIFFERENT configuration than:
- Simple grid layouts (zaburo)
- Egortrushin tessellation (0°/180° alternating)
- Random SA optimization

The key insight is that the 2-tree seed with ~75°/255° angles
creates an interlocking pattern that packs more efficiently.
""")

# Check if we have the jiweiliu kernel output
print("\nChecking for jiweiliu kernel solutions...")
jiweiliu_path = '/home/code/research/kernels/jiweiliu_super-fast-simulated-annealing-with-translations/'
if os.path.exists(jiweiliu_path):
    print(f"Kernel found at: {jiweiliu_path}")
    print("This kernel may produce better solutions for large N values.")